<a href="https://colab.research.google.com/github/mianyumifen-bot/codePublic/blob/main/%E4%B8%8B%E8%BD%BD%E6%95%B0%E6%8D%AE%E5%85%A8%E6%B5%81%E7%A8%8B%E6%95%B4%E5%90%882026_1_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 🛰️ 第一步：GEE 任务提交器 (运行此格提交下载任务)
# @markdown #### 1. 基础连接设置
project_id = "ee-mianyumifen"  # @param {type:"string"}
# 你的 SHP 文件路径
shp_path = "/content/drive/MyDrive/doverdesaltmarsh/doverdesaltmarsh.shp"  # @param {type:"string"}
# 中转站文件夹 (暂时存放压缩数据的地方)
staging_folder = "S2_Staging_Temp"  # @param {type:"string"}

# @markdown #### 2. 数据需求 (最易改的地方)
# @markdown *   想下什么就写什么，用逗号隔开。
# @markdown *   支持波段: B1-B12
# @markdown *   支持指数: NDVI, MNDWI, EVI, NDWI (代码里自动算)
bands_input = "B2, B3, B4, B8, MNDWI,B7"  # @param {type:"string"}

# @markdown #### 3. 时间与空间
start_year = 2016  # @param {type:"integer"}
end_year = 2026  # @param {type:"integer"}
buffer_radius = 700  # @param {type:"integer"}
scale = 10  # @param {type:"integer"}

# ================= 核心逻辑 (小白免进区) =================
import ee
import geemap
import geopandas as gpd
from shapely.geometry import mapping
from google.colab import drive

# 1. 环境初始化
drive.mount('/content/drive')
try:
    ee.Initialize(project=project_id)
except:
    ee.Authenticate()
    ee.Initialize(project=project_id)

# 2. 核心公式库 (易改性体现：想加新指数，就在这里加一行)
FORMULAS = {
    'NDVI':  '(b("B8") - b("B4")) / (b("B8") + b("B4"))',
    'MNDWI': '(b("B3") - b("B11")) / (b("B3") + b("B11"))',
    'NDWI':  '(b("B3") - b("B8")) / (b("B3") + b("B8"))',
    'EVI':   '2.5 * ((b("B8") - b("B4")) / (b("B8") + 6 * b("B4") - 7.5 * b("B2") + 1))'
}

def get_processed_collection(roi, year, needed_bands):
    """获取处理好的影像集合"""
    # 筛选
    col = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
            .filterBounds(roi)\
            .filterDate(f'{year}-01-01', f'{year}-12-31')

    # 预处理函数
    def process_img(img):
        # 1. 去云 (只用 QA60)
        qa = img.select('QA60')
        mask = qa.bitwiseAnd(1<<10).eq(0).And(qa.bitwiseAnd(1<<11).eq(0))

        # 2. 缩放与选波段 (只取出需要的原始波段进行缩放)
        # 找出 needed_bands 里所有 B 开头的波段
        raw_bands_to_select = [b for b in needed_bands if b.startswith('B')]
        # 无论如何，计算指数可能需要 B2,B3,B4,B8,B11，为了保险，先全拿出来缩放
        # 这里优化逻辑：先全部缩放，再选
        scaled = img.select(['B.*']).multiply(0.0001)

        # 3. 计算指数
        final_img = scaled
        for name, formula in FORMULAS.items():
            if name in needed_bands: # 只有用户点名要这个指数，我才算
                idx_img = scaled.expression(formula).rename(name)
                final_img = final_img.addBands(idx_img)

        # 4. 最终清洗：只保留用户清单里的波段
        return final_img.select(needed_bands).toFloat().updateMask(mask)

    return col.map(process_img)

def main_submit():
    # 解析用户需求
    target_bands = [b.strip() for b in bands_input.split(',')]
    print(f"🎯 任务目标: {target_bands}")

    # 读取站点
    gdf = gpd.read_file(shp_path)
    name_col = 'site' if 'site' in gdf.columns else ('name' if 'name' in gdf.columns else None)

    tasks_count = 0
    print("🚀 正在构建任务队列...")

    for idx, row in gdf.iterrows():
        site_name = str(row[name_col]) if name_col else f"site_{idx}"
        roi = ee.Geometry(mapping(row.geometry)).buffer(buffer_radius)

        for year in range(start_year, end_year + 1):
            col = get_processed_collection(roi, year, target_bands)

            if col.size().getInfo() == 0:
                print(f"  ⚠️ {site_name} - {year} 无影像，跳过")
                continue

            # 关键：为每张图打上标签 (Date_Tile) 用于后续拆分
            def add_index_tag(img):
                date = ee.Date(img.get('system:time_start')).format('YYYYMMdd')
                tile = ee.String(img.get('MGRS_TILE'))
                return img.set('system:index', date.cat('_').cat(tile))

            # 压扁成堆叠图 (Stack)
            stack = col.map(add_index_tag).toBands()

            # 导出
            task_name = f"{site_name}_{year}"
            task = ee.batch.Export.image.toDrive(
                image=stack,
                description=task_name,
                folder=staging_folder,
                fileNamePrefix=task_name,
                region=roi,
                scale=scale,
                maxPixels=1e13
            )
            task.start()
            tasks_count += 1
            print(f"  ✅ 提交: {site_name}_{year}")

    print(f"\n🎉 完成！共提交 {tasks_count} 个任务。")
    print("请去 GEE 网页端 Tasks 页面等待任务变成绿色 (COMPLETED)，然后再运行第二步。")

main_submit()

Mounted at /content/drive
🎯 任务目标: ['B2', 'B3', 'B4', 'B8', 'MNDWI', 'B7']
🚀 正在构建任务队列...
  ✅ 提交: doverdesaltmarsh_2016
  ✅ 提交: doverdesaltmarsh_2017
  ✅ 提交: doverdesaltmarsh_2018
  ✅ 提交: doverdesaltmarsh_2019
  ✅ 提交: doverdesaltmarsh_2020
  ✅ 提交: doverdesaltmarsh_2021
  ✅ 提交: doverdesaltmarsh_2022
  ✅ 提交: doverdesaltmarsh_2023
  ✅ 提交: doverdesaltmarsh_2024
  ✅ 提交: doverdesaltmarsh_2025
  ✅ 提交: doverdesaltmarsh_2026

🎉 完成！共提交 11 个任务。
请去 GEE 网页端 Tasks 页面等待任务变成绿色 (COMPLETED)，然后再运行第二步。


In [ ]:
# @title 📂 第二步：云端拆箱与整理 (GEE任务跑完后再运行我)
# @markdown #### 1. 路径设置
# 中转站 (必须和第一步一样)
staging_folder = "S2_Staging_Temp"  # @param {type:"string"}
# 最终成品仓库 (文件会完美分类存到这里)
final_output_folder = "S2_Final_Collection_doverdesaltmarsh"  # @param {type:"string"}

# @markdown #### 2. 功能开关
# 处理完是否删除中转站的压缩文件？(建议选是，节省空间)
delete_source_after_split = False  # @param {type:"boolean"}

# ================= 核心逻辑 (自动拆箱) =================
import os
import re
import rasterio
import shutil
from google.colab import drive
from tqdm.notebook import tqdm

drive.mount('/content/drive')
BASE_PATH = "/content/drive/MyDrive"
IN_DIR = os.path.join(BASE_PATH, staging_folder)
OUT_DIR = os.path.join(BASE_PATH, final_output_folder)

def split_and_organize():
    if not os.path.exists(IN_DIR):
        print(f"❌ 找不到中转文件夹: {IN_DIR}")
        return

    files = [f for f in os.listdir(IN_DIR) if f.endswith('.tif')]
    print(f"📦 发现 {len(files)} 个年合成文件，开始云端拆箱...\n")

    for filename in tqdm(files, desc="总进度"):
        src_path = os.path.join(IN_DIR, filename)

        # 解析文件名: site_year.tif
        try:
            site_name = filename.rsplit('_', 1)[0]
            year = filename.rsplit('_', 1)[1].replace('.tif', '')
        except:
            print(f"⚠️ 跳过命名格式不对的文件: {filename}")
            continue

        try:
            with rasterio.open(src_path) as src:
                descriptions = src.descriptions

                # 1. 建立索引：{ "日期_瓦片": { "波段名": 原始索引 } }
                # 目标是把乱序的堆叠波段按天归类
                grouped = {}
                for idx, desc in enumerate(descriptions):
                    if not desc: continue
                    # 匹配 GEE 导出的格式: 20200101_T12ABC_B8
                    # 注意：toBands() 的命名规则有时是 Date_Tile_Band，有时是 Index_Band
                    # 我们用正则通配
                    match = re.search(r'(\d{8})_(T\d{2}[A-Z]{3})_([a-zA-Z0-9]+)', desc)

                    if match:
                        d_str, t_str, b_name = match.groups()
                        key = f"{d_str}_{t_str}"
                        if key not in grouped: grouped[key] = {}
                        grouped[key][b_name] = idx + 1 # rasterio index start from 1

                if not grouped:
                    print(f"⚠️ {filename} 无法解析波段信息，可能是空图。")
                    continue

                # 2. 逐天导出小文件
                # 目标路径: Final/Site/Year/
                save_dir = os.path.join(OUT_DIR, site_name, year)
                os.makedirs(save_dir, exist_ok=True)

                for dt_key, bands_map in grouped.items():
                    date_val, tile_val = dt_key.split('_')

                    # 最终文件名: site_date_tile.tif
                    out_name = f"{site_name}_{date_val}_{tile_val}.tif"
                    out_path = os.path.join(save_dir, out_name)

                    # 准备写入
                    # 按波段名字母排序: B2, B3, NDVI...
                    sorted_bands = sorted(bands_map.keys())

                    meta = src.meta.copy()
                    meta.update({
                        'count': len(sorted_bands),
                        'driver': 'GTiff',
                        'compress': 'lzw' # 节省Drive空间
                    })

                    with rasterio.open(out_path, 'w', **meta) as dst:
                        for new_idx, b_name in enumerate(sorted_bands, start=1):
                            old_idx = bands_map[b_name]
                            data = src.read(old_idx)
                            dst.write(data, new_idx)
                            dst.set_band_description(new_idx, b_name) # 写入干净的名字

        except Exception as e:
            print(f"❌ 处理 {filename} 失败: {e}")
            continue

        # 3. 清理垃圾
        if delete_source_after_split:
            os.remove(src_path)
            print(f"🗑️ 已删除中转文件: {filename}")

    print(f"\n✅ 全部完成！你的数据已整齐存放在: {OUT_DIR}")

split_and_organize()